In [1]:
from collections.abc import Callable
from typing import cast
import sys, os, re

sys.path = (["../../src/", "../../"] if re.match(r"^(\w\:\\)|(/)", os.getcwd()) else []) + sys.path
import qubx # type: ignore

%qubxd 

%load_ext autoreload
%autoreload 2

import qubx.ta.indicators as ta # type: ignore
import qubx.pandaz.ta as pta # type: ignore
from qubx.core.basics import ( # type: ignore
    DataType,
    Deal,
    Instrument,
    Order,
    Position,
    Signal,
)
from qubx.core.interfaces import ( # type: ignore
    IStrategy,
    IStrategyContext,
    PositionsTracker,
    TriggerEvent,
)
from qubx.trackers.sizers import FixedLeverageSizer
from qubx import logger
from qubx.core.utils import recognize_time
from qubx.data.helpers import loader  # type: ignore
from qubx.core.lookups import lookup

from qubx.trackers.riskctrl import TrailingStopPositionTracker, SwingsStopLevels
import numpy as np
import pandas as pd


⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2025, ver. 0.6.72
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        


In [2]:
ld = loader("BINANCE.UM", "1h", source="csv::../../tests/data/csv_1h/", n_jobs=1)
data = ld["BTCUSDT", "2023-06-01":"2023-08-02"]

In [ ]:
# sw = pta.swings(data, pta.psar, iaf=0.02,maxaf=0.2)
# LookingGlass([ data, sw.trends ], backend="plotly").look().hover()

In [5]:
class GuineaPig(IStrategy):
    tests = {}

    def on_init(self, ctx: IStrategyContext) -> None:
        ctx.set_base_subscription(DataType.OHLC["1Min"])

    def on_fit(self, ctx: IStrategyContext):
        self.tests = {recognize_time(k): v for k, v in self.tests.items()}

    def on_event(self, ctx: IStrategyContext, event: TriggerEvent) -> list[Signal] | None:
        r = []
        ohlc = ctx.ohlc("BTCUSDT", self.timeframe, 20)
        for k in list(self.tests.keys()):
            if event.time >= k:
                s = self.tests.pop(k)
                match s:
                    case Signal():
                        r.append(s)
                    case (Signal(), Callable()):
                        r.append(s[0])
                        cast(Callable, s[1])(ctx, s[0])
                    case _:
                        logger.warning(f" - - - | {s} | - - - ")
        return r

In [6]:
I = lookup.find_symbol("BINANCE.UM", "BTCUSDT")

class SwingsLevels(GuineaPig):
    def tracker(self, ctx: IStrategyContext) -> PositionsTracker:
        return SwingsStopLevels("1h", 0.02, 0.2, FixedLeverageSizer(1.0), risk_controlling_side="client")

rep = simulate(
    strategies={
        "Trailing.Clent": SwingsLevels(
            tests={
                "2023-06-19 21:00:00": I.signal("2023-06-19 21:00:00", +1),
                "2023-07-05 01:00:00": I.signal("2023-07-05 01:00:00", -1),
            }
        ),
    },
    data={"ohlc(1h)": ld},
    capital=10000,
    instruments=["BINANCE.UM:BTCUSDT"],
    silent=True,
    debug="DEBUG",
    commissions="vip0_usdt",
    start="2023-06-01",
    stop="2023-08-01",
)

2025-08-26 21:35:17.622 [ 🐞 ] (runner) [Simulator] :: Preparing simulated trading on ['BINANCE.UM'] for 10000 USDT...
2025-08-26 21:35:17.623 [ ℹ️ ] (data) SimulatedDataProvider.BINANCE.UM is initialized
2025-08-26 21:35:17.624 [ 🐞 ] (runner) [simulator] :: Setting default schedule: 0 */1 * * *
2023-06-01 00:00:00.000 [🐞] [SimulationRunner] :: Running simulation from 2023-06-01 00:00:00 to 2023-08-01 00:00:00
2023-06-01 00:00:00.000 [ℹ️] SimulationRunner ::: Simulation started at 2023-06-01 00:00:00 :::
2023-06-01 00:00:00.000 [🐞]   [IteratedDataStreamsSlicer] :: Preloading initial data for ohlc.1Min 2023-06-01 00:00:00 : 2023-08-01 00:00:00 ...
2023-06-01 00:00:05.000 [ℹ️] All 1 instruments have data - strategy ready to start
2023-06-01 00:00:05.000 [🐞] [ProcessingManager] :: Invoking SwingsLevels on_fit
2023-06-01 00:00:05.000 [🐞] [ProcessingManager] :: SwingsLevels is fitted
2023-06-01 01:00:00.000 [❌] Strategy SwingsLevels raised an exception: 'SwingsLevels' object has no attribute

2023-06-01 04:00:00.000 [❌] Strategy SwingsLevels raised an exception: 'SwingsLevels' object has no attribute 'timeframe'
2023-06-01 04:00:00.000 [❌] Traceback (most recent call last):
  File "/home/andrij/devs/Qubx/experiments/trackers/../../src/qubx/core/mixins/processing.py", line 314, in __process_data
    signals.extend(self._as_list(self._strategy.on_event(self._context, _trigger_event)))
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_3172339/2386609949.py", line 12, in on_event
    ohlc = ctx.ohlc("BTCUSDT", self.timeframe, 20)
                               ^^^^^^^^^^^^^^
AttributeError: 'SwingsLevels' object has no attribute 'timeframe'

2023-06-01 05:00:00.000 [❌] Strategy SwingsLevels raised an exception: 'SwingsLevels' object has no attribute 'timeframe'
2023-06-01 05:00:00.000 [❌] Traceback (most recent call last):
  File "/home/andrij/devs/Qubx/experiments/trackers/../../src/qubx/core/mixins/processing.py", l

In [ ]:
rep[0].chart_signals("BTCUSDT", ld, start="2023-06-18", end="2023-07-10", 
                    overlay=[pta.swings(data, pta.psar, iaf=0.02, maxaf=0.2).trends],
                     show_signals=True)